# Homolog data munging

Generate fasta files for each tested sequence of closest homologs from the MSA used to infer the EVcouplings model. 

In [1]:
import os, sys
import scipy
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib import rc

mpl.rcParams['pdf.fonttype'] = 42

from matplotlib.ticker import ScalarFormatter
%matplotlib inline

## Constants

In [2]:
#############################
#                           #
#                           #
#        CONSTANTS          #
#                           #
#                           #
#############################
LOCAL_DIR = '.'
DATA_DIR = LOCAL_DIR+'/data'

OUTPUT_DIR = LOCAL_DIR+'/figures/homolog_msas'
if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

In [3]:
from importlib import reload
if LOCAL_DIR not in sys.path: 
    sys.path.append(LOCAL_DIR) #Helper Functions Here
import FramHelperScripts
reload(FramHelperScripts)
FHS = FramHelperScripts.FramHelperFunctions(DATA_DIR)

## Load Data

In [4]:
#load all tested sequences
sequences = FHS.get_sequences_df()[['manuscript_name', 'full_sequence',]]

#load the natural MSA used to generate the model
msa_df = FHS.get_natural_msa_df(remove_unaligned_positions=False)

#remove non-existant samples from sample order
sample_order = FHS.get_sample_order(sequences.manuscript_name)

WT_SEQ = sequences[sequences.manuscript_name=='WT TEM-1'].iloc[0].full_sequence

In [5]:
def get_mut_count(seq1, seq2):
    return np.sum(
        [1 for idx in range(0, len(seq1)) if seq1[idx] != seq2[idx]]
    )

def get_seq_id_w_muts(seq_id, target_id, mutcount_to_target, mutcount_to_wt):
    return '{0} | {1:.0f}mut to {2} | {3:.0f}mut to WT'.format(
        seq_id, mutcount_to_target, target_id, mutcount_to_wt
    )

def output_msa_nearest_homologs(
    target_tested_seqname, homologs_to_include=5, outfile=None
):
    target_seq = sequences[
        sequences.manuscript_name == target_tested_seqname
    ].iloc[0].full_sequence
    
    #remove WT from comparison - we want top homologs that are not wt
    msa_w_mutcount_df = msa_df[msa_df.seq_id != 'BLAT_ECOLX/24-286'].assign(
        mutcount = lambda df: df.seq.apply(
            lambda msa_seq: get_mut_count(target_seq, msa_seq)
        )
    ).sort_values(
        by='mutcount'
    ).reset_index(
        drop=True
    )[0:homologs_to_include]
    
    fasta_entries = [
        '>{0}\n{1}'.format(target_tested_seqname, target_seq),
        '>{0}\n{1}'.format(
            get_seq_id_w_muts(
                'WT TEM-1', 
                target_tested_seqname,
                get_mut_count(target_seq, WT_SEQ),
                get_mut_count(WT_SEQ, WT_SEQ)
            ), 
            WT_SEQ
        ),
    ]
    for idx, row in msa_w_mutcount_df.iterrows():
        fasta_entries.append(
            '>{0}\n{1}'.format(
                get_seq_id_w_muts(
                    row.seq_id, 
                    target_tested_seqname,
                    row.mutcount, 
                    get_mut_count(row.seq, WT_SEQ)
                ), 
                row.seq
            )
        )
    output = '\n\n'.join(fasta_entries)
    if outfile == None: print(output)
    else: 
        f = open(outfile, 'w')
        f.write(output)
        f.close()

In [6]:
for seq in sample_order:
    output_msa_nearest_homologs(
        seq, 
        outfile=OUTPUT_DIR+'/{0}.fa'.format(seq)
    )